In [ ]:
import sys 
import numpy as np 
sys.path.append('/home/ak/Documents/Research/QFPaper/Code/')
import hmm 
import pandas as pd
import plotly
import plotly.plotly as py
import seaborn as sns
from hmm import hmm_calibration
from observation_models import *
import matplotlib.pyplot as plt 
%matplotlib inline 
import copy

prng = np.random.RandomState(10)
n_components = 3
startprob = prng.rand(n_components)
startprob = startprob / startprob.sum()
transmat = prng.rand(n_components, n_components) #random starting point
transmat /= np.tile(transmat.sum(axis=1)[:, np.newaxis], (1, n_components))#normalising so all entries sum up to 1

sigmas = [ .3, 0.8, 1][0:n_components]
lambdas = [0.5, 5., 0.8][0:n_components]
weights = [0, 0, 0][0:n_components]
priors = {'sigmas':[0.3, .85, 1.05][0:n_components], 'lambdas':[0.05, 0.9, 0.72][0:n_components], 
          'weights':[0.1, 0.02, 0.03][0:n_components]}
sample_size = 500
#generate state and observation data from the \"right\" distribution\n",
states = hmm_calibration.sample_states(startprob, transmat, prng, sample_size=sample_size)

observation_data=Exp_UniGauss.sample_data(states,lambdas, sigmas, weights, 0, rng=prng)
data_set = pd.DataFrame()
data_set['tau'] = observation_data[0]
data_set['price_change'] = observation_data[1]
#define the calibrator and model 
update_tag = 'tpsl'

obs_model = Exp_UniGauss(n_components, epsilon=0)
the_hmm = hmm_calibration(obs_model, n_components, max_iter=10) 
result = the_hmm.run_hmm(data_set, update_tag=update_tag, initialise_method_tag='uniform',priors=priors, rng=prng)